分类模型的损失函数有很多种,比如
- svm的hinge loss
- 逻辑回归的logit loss
- MSE
- 多分类时的交叉熵

# 多分类模型的设定

多分类模型下,一共有K个类,真实概率分布为$(y^1, y^2, ..., y^K)$,预测概率分布为$(\hat{y}^1, \hat{y}^2, ..., \hat{y}^K) $.样本量为N.

# 均方损失/MSE (Mean Square Error)
基于MSE的损失函数为  
$$L = \frac{1}{N} \sum \limits_{i=1}^{N} ||y_i - \hat{y}_i ||_2^2  $$
其中$y_i和\hat{y}_i$分别是第i个样例的真实值和预测值,可以是向量. $|| a - b ||_2^2$表示a和b的[L2距离或欧几里得距离](https://en.wikipedia.org/wiki/Euclidean_distance).

多分类模型下损失函数则为
$$L = \frac{1}{N} \sum \limits_{i=1}^{N} \sum \limits_{k=1}^K  (y_i^k - \hat{y}_i^k)^2  $$


# 交叉熵方法 (Cross-entropy method)
交叉熵方法基于信息论中[交叉熵](https://en.wikipedia.org/w/index.php?title=Cross_entropy&oldid=491011306)的概念,反映预测概率分布和真实概率分布的差别.  

多分类模型下,基于交叉熵的损失函数为
$$ L = \frac{1}{N} \sum \limits_{i=1}^{N} \sum \limits_{k=1}^{K} \hat{y}_i^k * log \hat{y}_i^k $$

往往每个样例的真实概率分布为one-hot型,也就是说每个样例都属于某类.所以上式等于
$$ \frac{1}{N} \sum \limits_{i=1}^{N} log \hat{y}_i^{k_i} $$
其中$k_i$为第i个样例的类别.  
在二分类模型中,不妨说类别是1和0, 也就是$y \in \{1, 0\}$.一般来说,模型的预测值是样例为1类的预测概率.这样当$k_i=1时,y_i=1, \hat{y}_i^{k_i} = log \hat{y}_i;当y_i=0时,y_i=0,\hat{y}_i^{k_i} = log (1-\hat{y}_i)$.不论$k_i$是0还是1, $\hat{y}_i^{k_i} = y_i*log\hat{y}_i + (1-y_i) * log (1-\hat{y_i})$
上式就等于
$$  \frac{1}{N} \sum \limits_{i=1}^{N} y_i*log\hat{y}_i + (1-y_i) * log (1-\hat{y_i})$$